# Data Cleaning

In [1]:
# import modules
import pandas as pd

In [2]:
# load data
dist_ctrs = pd.read_csv("../data/raw/distribution_centers.csv")
events = pd.read_csv("../data/raw/events.csv")
inv_items = pd.read_csv("../data/raw/inventory_items.csv")
order_items = pd.read_csv("../data/raw/order_items.csv")
orders = pd.read_csv("../data/raw/orders.csv")
products = pd.read_csv("../data/raw/products.csv")
users = pd.read_csv("../data/raw/users.csv")

## Distribution Centers Table

In [3]:
dist_ctrs.head()

,id,name,latitude,longitude
0,1,Memphis TN,35.1174,-89.9711
1,2,Chicago IL,41.8369,-87.6847
2,3,Houston TX,29.7604,-95.3698
3,4,Los Angeles CA,34.0500,-118.2500
4,5,New Orleans LA,29.9500,-90.0667


### Observations

In [4]:
f"Num rows: {dist_ctrs.shape[0]}"

'Num rows: 10'

In [5]:
# check data types
dist_ctrs.dtypes

id             int64
name          object
latitude     float64
longitude    float64
dtype: object

In [6]:
# check missing values
dist_ctrs.isna().sum()

id           0
name         0
latitude     0
longitude    0
dtype: int64

### Actions

In [7]:
# rename New York distribution center name
dist_ctrs['name'] = dist_ctrs['name'].replace('Port Authority of New York/New Jersey NY/NJ', 'New York NY')
dist_ctrs

,id,name,latitude,longitude
0,1,Memphis TN,35.1174,-89.9711
1,2,Chicago IL,41.8369,-87.6847
2,3,Houston TX,29.7604,-95.3698
3,4,Los Angeles CA,34.0500,-118.2500
4,5,New Orleans LA,29.9500,-90.0667
5,6,New York NY,40.6340,-73.7834
6,7,Philadelphia PA,39.9500,-75.1667
7,8,Mobile AL,30.6944,-88.0431
8,9,Charleston SC,32.7833,-79.9333
9,10,Savannah GA,32.0167,-81.1167


In [8]:
# split name into seperate city and state columns
dist_ctrs[['city', 'state']] = dist_ctrs['name'].str.rsplit(pat=' ', n=1, expand=True)
dist_ctrs = dist_ctrs.drop(columns='name')[['id', 'city', 'state', 'latitude', 'longitude']]
dist_ctrs.head()

,id,city,state,latitude,longitude
0,1,Memphis,TN,35.1174,-89.9711
1,2,Chicago,IL,41.8369,-87.6847
2,3,Houston,TX,29.7604,-95.3698
3,4,Los Angeles,CA,34.0500,-118.2500
4,5,New Orleans,LA,29.9500,-90.0667


## Events Table

In [9]:
events.head()

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
0,2198523,NaN,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel
1,1773216,NaN,3,7a3fc3f2-e84f-44fe-8876-eff76741f7a3,2020-08-07 08:41:00+00:00,85.114.141.79,Santa Isabel,São Paulo,07500-000,Safari,Adwords,/cancel,cancel
2,2380515,NaN,3,13d9b2fb-eee1-43fd-965c-267b38dd7125,2021-02-15 18:48:00+00:00,169.250.255.132,Mairiporã,São Paulo,07600-000,IE,Adwords,/cancel,cancel
3,2250597,NaN,3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00,137.25.222.160,Cajamar,São Paulo,07750-000,Chrome,Adwords,/cancel,cancel
4,1834446,NaN,3,d09dce10-a7cb-47d3-a9af-44975566fa03,2019-09-05 01:18:00+00:00,161.114.4.174,São Paulo,São Paulo,09581-680,Chrome,Email,/cancel,cancel


### Observations

In [10]:
f"Num rows: {events.shape[0]}"

'Num rows: 2431963'

In [11]:
# check data types
events.dtypes

id                   int64
user_id            float64
sequence_number      int64
session_id          object
created_at          object
ip_address          object
city                object
state               object
postal_code         object
browser             object
traffic_source      object
uri                 object
event_type          object
dtype: object

In [12]:
# check missing values
events.isna().sum()

id                       0
user_id            1125671
sequence_number          0
session_id               0
created_at               0
ip_address               0
city                 23080
state                    0
postal_code              0
browser                  0
traffic_source           0
uri                      0
event_type               0
dtype: int64

### Actions

In [13]:
# set created_at data type to datetime
events['created_at'] = pd.to_datetime(events['created_at'].str[:19])

In [14]:
# attempted to identify missing user_ids from other events from the same session
missing_user_id_events = events[events['user_id'].isna()]
present_user_id_events = events[~events['user_id'].isna()]

# no overlapping sessions where found
len(set(missing_user_id_events['session_id']) & set(present_user_id_events['session_id']))

0

### Impact

In [15]:
# percentage of missing values
events.isna().sum() / events.shape[0] * 100

id                  0.000000
user_id            46.286518
sequence_number     0.000000
session_id          0.000000
created_at          0.000000
ip_address          0.000000
city                0.949028
state               0.000000
postal_code         0.000000
browser             0.000000
traffic_source      0.000000
uri                 0.000000
event_type          0.000000
dtype: float64

## Inventory Items Table

In [16]:
inv_items.head()

,id,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,67971,13844,2022-07-02 07:09:20+00:00,2022-07-24 06:33:20+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
1,67972,13844,2023-12-20 03:28:00+00:00,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
2,67973,13844,2023-06-04 02:53:00+00:00,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
3,72863,13844,2021-10-16 22:58:52+00:00,2021-11-22 02:19:52+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
4,72864,13844,2021-08-07 16:33:00+00:00,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7


### Observations

In [17]:
f"Num rows: {inv_items.shape[0]}"

'Num rows: 490705'

In [18]:
# check data types
inv_items.dtypes

id                                  int64
product_id                          int64
created_at                         object
sold_at                            object
cost                              float64
product_category                   object
product_name                       object
product_brand                      object
product_retail_price              float64
product_department                 object
product_sku                        object
product_distribution_center_id      int64
dtype: object

In [19]:
# check missing values
inv_items.isna().sum()

id                                     0
product_id                             0
created_at                             0
sold_at                           308946
cost                                   0
product_category                       0
product_name                          29
product_brand                        401
product_retail_price                   0
product_department                     0
product_sku                            0
product_distribution_center_id         0
dtype: int64

### Actions

In [20]:
# set created_at and sold_at columns to date-time data type
inv_items['created_at'] = pd.to_datetime(inv_items['created_at'].str[:19])
inv_items['sold_at'] = pd.to_datetime(inv_items['sold_at'].str[:19])

In [21]:
# round cost and product_retail price to consistent 2 decimal placesproduct_retail_price
inv_items['product_retail_price'] = inv_items['product_retail_price'].round(2)
inv_items['cost'] = inv_items['cost'].round(2)

In [22]:
# attempted to recover missing product and brand name values from product table
products_with_names = set(products[~products['name'].isna()]['id'])
inv_items_without_names = set(inv_items[inv_items['product_name'].isna()]['product_id'])

# no missing names found in products table
len(products_with_names & inv_items_without_names)

0

### Impact

In [23]:
# percentage of missing values
# note: missing sold_at rows are from products that haven't been sold yet
inv_items.isna().sum() / inv_items.shape[0] * 100

id                                 0.000000
product_id                         0.000000
created_at                         0.000000
sold_at                           62.959619
cost                               0.000000
product_category                   0.000000
product_name                       0.005910
product_brand                      0.081719
product_retail_price               0.000000
product_department                 0.000000
product_sku                        0.000000
product_distribution_center_id     0.000000
dtype: float64

## Order Items Table

In [24]:
order_items.head()

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaN,NaN,NaN,0.02
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaN,0.02
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,0.02
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19+00:00,2023-12-24 16:44:00+00:00,NaN,NaN,0.02
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,2022-06-19 19:29:00+00:00,NaN,NaN,0.02


### Observations

In [25]:
f"Num rows: {order_items.shape[0]}"

'Num rows: 181759'

In [26]:
# check data types
order_items.dtypes

id                     int64
order_id               int64
user_id                int64
product_id             int64
inventory_item_id      int64
status                object
created_at            object
shipped_at            object
delivered_at          object
returned_at           object
sale_price           float64
dtype: object

In [27]:
# check missing values
# note: all missing values are expected here and align with status
order_items.isna().sum()

id                        0
order_id                  0
user_id                   0
product_id                0
inventory_item_id         0
status                    0
created_at                0
shipped_at            63478
delivered_at         117918
returned_at          163527
sale_price                0
dtype: int64

In [28]:
# incorrect date ordering
sum(order_items['created_at'] > order_items['shipped_at'])

35872

### Actions

In [29]:
# update date columns to date-time
order_items['created_at'] = pd.to_datetime(order_items['created_at'].str[:19])
order_items['shipped_at'] = pd.to_datetime(order_items['shipped_at'].str[:19])
order_items['delivered_at'] = pd.to_datetime(order_items['delivered_at'].str[:19])
order_items['returned_at'] = pd.to_datetime(order_items['returned_at'].str[:19])

In [30]:
# round sale_price to 2 decimal places
order_items['sale_price'] = order_items['sale_price'].round(2)

### Impact

In [31]:
# percentage of incorrect created_at and shipped_at rows
sum(order_items['created_at'] > order_items['shipped_at']) / order_items.shape[0] * 100

19.736024075836685

## Orders Table

In [32]:
orders.head()

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,8,5,Cancelled,F,2022-10-20 10:03:00+00:00,NaN,NaN,NaN,3
1,60,44,Cancelled,F,2023-01-20 02:12:00+00:00,NaN,NaN,NaN,1
2,64,46,Cancelled,F,2021-12-06 09:11:00+00:00,NaN,NaN,NaN,1
3,89,65,Cancelled,F,2020-08-13 09:58:00+00:00,NaN,NaN,NaN,1
4,102,76,Cancelled,F,2023-01-17 08:17:00+00:00,NaN,NaN,NaN,2


### Observations

In [33]:
f"Num rows: {orders.shape[0]}"

'Num rows: 125226'

In [34]:
# check data types
orders.dtypes

order_id         int64
user_id          int64
status          object
gender          object
created_at      object
returned_at     object
shipped_at      object
delivered_at    object
num_of_item      int64
dtype: object

In [35]:
# check missing values
orders.isna().sum()

order_id             0
user_id              0
status               0
gender               0
created_at           0
returned_at     112696
shipped_at       43765
delivered_at     81342
num_of_item          0
dtype: int64

### Actions

In [36]:
# update date columns to date-time
orders['created_at'] = pd.to_datetime(orders['created_at'].str[:19])
orders['shipped_at'] = pd.to_datetime(orders['shipped_at'].str[:19])
orders['delivered_at'] = pd.to_datetime(orders['delivered_at'].str[:19])

## Products Table

In [37]:
products.head()

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,13842,2.51875,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women,EBD58B8A3F1D72F4206201DA62FB1204,1
1,13928,2.33835,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women,2EAC42424D12436BDD6A5B8A88480CC3,1
2,14115,4.87956,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women,EE364229B2791D1EF9355708EFF0BA34,1
3,14157,4.64877,Accessories,Enzyme Regular Solid Army Caps-Olive W35S45D (...,MG,10.99,Women,00BD13095D06C20B11A2993CA419D16B,1
4,14273,6.50793,Accessories,Washed Canvas Ivy Cap - Black W11S64C,MG,15.99,Women,F531DC20FDE20B7ADF3A73F52B71D0AF,1


### Observations

In [38]:
f"Num rows: {products.shape[0]}"

'Num rows: 29120'

In [39]:
# check data types
products.dtypes

id                          int64
cost                      float64
category                   object
name                       object
brand                      object
retail_price              float64
department                 object
sku                        object
distribution_center_id      int64
dtype: object

In [40]:
# check missing values
products.isna().sum()

id                         0
cost                       0
category                   0
name                       2
brand                     24
retail_price               0
department                 0
sku                        0
distribution_center_id     0
dtype: int64

### Actions

In [41]:
# rounding number of decimals in cost and retail price
products['retail_price'] = products['retail_price'].round(2)
products['cost'] = products['cost'].round(2)

In [42]:
# attempted to recover product names from inventory items table
product_ids_with_names = inv_items[~inv_items['product_name'].isna()]['product_id']
product_ids_without_names = products[products['name'].isna()]['id']

# no missing product names found
len(set(product_ids_with_names) & set(product_ids_without_names))

0

In [43]:
# attempted to recover brand names from inventory items table
product_ids_with_brands = inv_items[~inv_items['product_brand'].isna()]['product_id']
product_ids_without_brands = products[products['brand'].isna()]['id']

# no missing brands found
len(set(product_ids_with_brands) & set(product_ids_without_brands))

0

### Impact

In [44]:
# percentage of missing name and brand values
products.isna().sum() / products.shape[0] * 100

id                        0.000000
cost                      0.000000
category                  0.000000
name                      0.006868
brand                     0.082418
retail_price              0.000000
department                0.000000
sku                       0.000000
distribution_center_id    0.000000
dtype: float64

### Users Table

In [45]:
users.head()

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
0,457,Timothy,Bush,timothybush@example.net,65,M,Acre,87620 Johnson Hills,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2022-07-19 13:51:00+00:00
1,6578,Elizabeth,Martinez,elizabethmartinez@example.com,34,F,Acre,1705 Nielsen Land,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2023-11-08 18:49:00+00:00
2,36280,Christopher,Mendoza,christophermendoza@example.net,13,M,Acre,125 Turner Isle Apt. 264,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Email,2019-08-24 06:10:00+00:00
3,60193,Jimmy,Conner,jimmyconner@example.com,64,M,Acre,0966 Jose Branch Apt. 008,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2020-02-15 11:26:00+00:00
4,64231,Natasha,Wilson,natashawilson@example.net,25,F,Acre,20798 Phillip Trail Apt. 392,69917-400,Rio Branco,Brasil,-9.945568,-67.83561,Search,2020-03-13 06:45:00+00:00


### Observations

In [46]:
f"Num rows: {users.shape[0]}"

'Num rows: 100000'

In [47]:
# check data types
users.dtypes

id                  int64
first_name         object
last_name          object
email              object
age                 int64
gender             object
state              object
street_address     object
postal_code        object
city               object
country            object
latitude          float64
longitude         float64
traffic_source     object
created_at         object
dtype: object

In [48]:
# check missing values
users.isna().sum()

id                  0
first_name          0
last_name           0
email               0
age                 0
gender              0
state               0
street_address      0
postal_code         0
city              958
country             0
latitude            0
longitude           0
traffic_source      0
created_at          0
dtype: int64

### Actions

In [49]:
# update created_at to date-time
users['created_at'] = pd.to_datetime(users['created_at'].str[:19])

In [50]:
# try to recover city values from events table
user_ids_with_city = events[~events['city'].isna()]['user_id']
user_ids_without_city = users[users['city'].isna()]['id']

# no city values recovered
len(set(user_ids_with_city) & set(user_ids_without_city))

0

### Impact

In [51]:
users.isna().sum() / 100000 * 100

id                0.000
first_name        0.000
last_name         0.000
email             0.000
age               0.000
gender            0.000
state             0.000
street_address    0.000
postal_code       0.000
city              0.958
country           0.000
latitude          0.000
longitude         0.000
traffic_source    0.000
created_at        0.000
dtype: float64

## Saving cleaned data

In [52]:
# saving cleaned data
dist_ctrs.to_parquet("../data/clean/distribution_centers.parquet")
events.to_parquet("../data/clean/events.parquet")
inv_items.to_parquet("../data/clean/inventory_items.parquet")
order_items.to_parquet("../data/clean/order_items.parquet")
orders.to_parquet("../data/clean/orders.parquet")
products.to_parquet("../data/clean/products.parquet")
users.to_parquet("../data/clean/users.parquet")